### 0. Install and load library

In [ ]:
#Restart kernel after installation: Runtime -> Restart runtime

#!pip install -U transformers sentencepiece datasets

In [1]:
from datasets import load_dataset
from datasets import DatasetDict
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification, pipeline
import tensorflow as tf
from tqdm import tqdm

### 1. Load dataset

In [2]:
datasets = load_dataset("ag_news")

Using custom data configuration default


Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


### 2. Split dataset

In [3]:
datasets.keys() #dict_keys(['train', 'test'])

# split original train set into train/val with 8/2 ratio
# gather together to process splits at once
train_valid = datasets['train'].train_test_split(test_size=.2)
train_valid_test_datasets = DatasetDict({
    'train': train_valid['train'],
    'valid': train_valid['test'],
    'test': datasets['test']
})

In [4]:
# ['World', 'Sports', 'Business', 'Sci/Tech']
dataset_labels = datasets['train'].features['label'].names
id2label = {id: label for id, label in enumerate(dataset_labels)}
label2id = {label: id for id, label in id2label.items()}

### 3. Load model and tokenizer with correct configs 

In [5]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(dataset_labels), label2id=label2id, id2label=id2label)
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

### 4. Tokenize and convert dataset to DataLoader

In [6]:
encoded_datasets = train_valid_test_datasets.map(lambda examples: tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True)
encoded_datasets = encoded_datasets.map(lambda examples: {'labels': examples['label']}, batched=True)
encoded_datasets.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'labels'])

train_features = {x: encoded_datasets['train'][x].to_tensor(default_value=0, shape=[None, tokenizer.model_max_length]) for x in ['input_ids', 'attention_mask']}
val_features = {x: encoded_datasets['valid'][x].to_tensor(default_value=0, shape=[None, tokenizer.model_max_length]) for x in ['input_ids', 'attention_mask']}
test_features = {x: encoded_datasets['test'][x].to_tensor(default_value=0, shape=[None, tokenizer.model_max_length]) for x in ['input_ids','attention_mask']}

train_tfdataset = tf.data.Dataset.from_tensor_slices((train_features, encoded_datasets['train']['labels'])).batch(8).prefetch(tf.data.AUTOTUNE)
val_tfdataset = tf.data.Dataset.from_tensor_slices((val_features, encoded_datasets['valid']['labels'])).batch(8).prefetch(tf.data.AUTOTUNE)
test_tfdataset = tf.data.Dataset.from_tensor_slices((test_features, encoded_datasets['test']['labels'])).batch(8).prefetch(tf.data.AUTOTUNE)

### 5. Prepare training arguments

In [13]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
metrics = ['accuracy']
epochs = 3

### 6. Training

In [ ]:
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model.fit(train_tfdataset,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_tfdataset
                    )

### 7. Predicts on Test set

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(test_tfdataset, verbose=1)
y_pred = np.argmax(y_pred.logits, axis=1)

y_true = []
for element in test_samples.as_numpy_iterator(): 
  y_true.append(element[1])
y_true = np.concatenate(y_true)


print(classification_report(y_true, y_pred))

### 8. Inference using pipeline

In [52]:
ag_news_pipeline = pipeline('text-classification', model=model, tokenizer=tokenizer)

In [ ]:
"""
datasets['test'][0]

>>> {
    'label': 2,
    'text': "Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul."
    }

"""

In [55]:
test_input = datasets['test'][0]['text']
outputs = ag_news_pipeline(test_input)
print(outputs)

"""
>>> [{'label': 'LABEL_2', 'score': 0.9857332110404968}]
"""

[{'label': 'Sports', 'score': 0.25402387976646423}]


"\n>>> [{'label': 'LABEL_2', 'score': 0.9857332110404968}]\n"